# Producer Development


### Notebook for testing out the interface to Twitter and IEX

In [76]:
#Import
import numpy as np
import pandas as pd

#Twitter requirements
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

#IEX requirements
import urllib.request


import json

### Create Company List

In [77]:
companies = {
    "Apple":"AAPL",
    "Facebook":"FB",
    "Google":"GOOG"
    }

#company_df = pd.DataFrame(companies.items(), columns=['Company Name','Ticker'])
company_df = pd.DataFrame.from_dict(companies, orient='index')
company_df.index.name = 'Company'
company_df.columns=['Ticker']
company_df.head()

,Ticker
Company,
Apple,AAPL
Facebook,FB
Google,GOOG


In [30]:
#Write to CSV
company_df.to_csv('companies.csv')

### Twitter Ingestion

In [78]:
#Created a twitter tokens file. That has the tokens. This will not be in GIT
from twitter_tokens import *

In [79]:
#Add Code to read CSV file. In order to account for changes in the list, have to restart
companies = pd.read_csv('companies.csv')

#Add code to add ticker symbol
companies['tweet_ticker']=companies['Ticker'].apply(lambda x: '$'+x)
companies

,Company,Ticker,tweet_ticker
0,Apple,AAPL,$AAPL
1,Facebook,FB,$FB
2,Google,GOOG,$GOOG


In [80]:
#tickers = companies['tweet_ticker'].tolist()
tickers = companies['tweet_ticker'].tolist()
#Tweets with the ticker in front come in very slowly. May take a while to build up. But these are official tweets

In [82]:
#Create Filter function to filter attributes and Add Company name to the dictionary.

#Want to create a simple list, but how do I handle the users sub?
attributes = ['created_at',
             'id_str',
             'text',
              'quote_count',
              'reply_count',
              'retweet_count',
              'favorite_count',
              'retweeted',
              ['user','name'],
              ['user','followers_count'],
              ['user','statuses_count'],
              ['entities','hashtags'],
              ['entities','symbols']
               ]
def filter_attr(data):
    output = {}
    for element in attributes:
        if isinstance(element, str):
            output[element]=data[element]
        else:
            string = str(element[0])+'_'+str(element[1])
            #output[element[1]]=data[element[0]][element[1]]
            output[string]=data[element[0]][element[1]]
  
    #Need to also add the company name to output dictionary.
    search = data['text']
    return output

#This is a basic listener that just prints received tweets to stdout.
class TweetListener(StreamListener):
    global count
    count = 0
    def on_data(self, data):
        try:
            datajson = json.loads(data)
            filtered = filter_attr(datajson)
            global count
            count += 1
            print(count)
            print(filtered)
            #print(datajson['text'])
            #coll_reference.insert_one(datajson)
            return True
        except Exception as e:
            print(e)

    def on_error(self, status):
        print('The error code is: ' + repr(status))
        #Continue even if there is an error
        return True

#This handles Twitter authetication and the connection to Twitter Streaming API
tweetlist = TweetListener(api=API(wait_on_rate_limit=True,wait_on_rate_limit_notify=True))
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
stream = Stream(auth, tweetlist)

#Filters by the ticker names
print('Filtering: ' + str(tickers))
stream.filter(track=tickers)

Filtering: ['$AAPL', '$FB', '$GOOG']
1
{'created_at': 'Fri Jan 26 02:09:58 +0000 2018', 'id_str': '956710776143863809', 'text': "RT @SharePlanner: Questions about trading? Shoot them my way, and I'd be glad to answer anything you want answered #stocks $AAPL $FB $NFLX…", 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'user_name': '7animal', 'user_followers_count': 1534, 'user_statuses_count': 5566, 'entities_hashtags': [{'text': 'stocks', 'indices': [115, 122]}], 'entities_symbols': [{'text': 'AAPL', 'indices': [123, 128]}, {'text': 'FB', 'indices': [129, 132]}]}
2
{'created_at': 'Fri Jan 26 02:10:00 +0000 2018', 'id_str': '956710784914075650', 'text': 'RT @PlanetZiggurat: Our SMS campaign was successfully launched  https://t.co/SEWT8fznze \n#Mexico #australia #chile #peru #bolivia #columbia…', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'retweeted': False, 'user_name': 'Laura Morales', 'user_followers_co

KeyboardInterrupt: 